# Trading and investment strategy application
## Problem Definition
You have developed an application to learn trading and investment strategy and you want to build a database to show records of the selling price of a virtual financial instrument in your application called meta-stocks. In your application, there are different meta-stocks markets, and for each meta-stock market you want to store the name and the acronym. Within a meta-stock market, there are several meta-stocks representing ownership on virtual businesses. For each meta-stock, you want to store the name and the acronym. You also would like to store information about users in your application, like the username and email address. Finally, you want to store in a table called meta-records timestamped records of buying and selling operations. For each record, you need to store a timestamp indicating the time and date of the operation, the selling price of the operation, a references to the meta-stock, and references to the buyer (user that bought the meta-stock) and the seller (user that sold the meta-stock). 
Design a database to support this fascinating application 


## Solution

```sql
# First let´s create the database
CREATE DATABASE meta-stocks

# Use the database we just created
USE meta-stocks

# Entity meta-stock market


```